In [ ]:
import snorkel


In [ ]:
from spikeQ import query_spike

In [ ]:
df = query_spike(["T-cell exhaustion"], results_limit=1000)

Running query: T-cell exhaustion
{'Date': 'Sun, 21 Mar 2021 13:57:20 GMT', 'Connection': 'keep-alive', 'Location': '/api/3/search/query/f8073fbcc4b527872820e3fc62abe053/browse', 'Explain-Location': '/api/3/search/query/f8073fbcc4b527872820e3fc62abe053/explain', 'Stream-Location': '/api/3/search/query/f8073fbcc4b527872820e3fc62abe053/stream', 'CSV-Location': '/api/3/search/query/f8073fbcc4b527872820e3fc62abe053/csv', 'Cache-Control': 'no-cache, no-store, must-revalidate', 'Expires': '0', 'Strict-Transport-Security': 'max-age=15724800'}


In [ ]:
%%bash
#rm -rf "/home/ubuntu/.local/lib/python3.6/site-packages/en_ner_*"

In [ ]:
import stanza
import spacy_stanza
import scispacy
import spacy

from scispacy.linking import EntityLinker
from scispacy.abbreviation import AbbreviationDetector
from scispacy.hyponym_detector import HyponymDetector

In [ ]:
# download and initialize a mimic pipeline with an i2b2 NER model
# stanza.download('en', package='mimic', processors={'ner': 'i2b2'})
# stanza.download('en', package='craft')
stanza.download('en', package='genia', processors={'ner':'bionlp13cg'})
nlp = spacy_stanza.load_pipeline('en', package='genia', processors={'ner': 'bionlp13cg'}, use_gpu=True)
nlp.add_pipe("abbreviation_detector")
nlp.add_pipe("hyponym_detector", last=True, config={"extended": False})



2021-03-21 13:57:23 INFO: Downloading these customized packages for language: en (English)...
| Processor       | Package    |
--------------------------------
| tokenize        | genia      |
| pos             | genia      |
| lemma           | genia      |
| depparse        | genia      |
| ner             | bionlp13cg |
| pretrain        | genia      |
| forward_charlm  | pubmed     |
| backward_charlm | pubmed     |

2021-03-21 13:57:23 INFO: File exists: /home/ubuntu/stanza_resources/en/tokenize/genia.pt.
2021-03-21 13:57:24 INFO: File exists: /home/ubuntu/stanza_resources/en/pos/genia.pt.
2021-03-21 13:57:24 INFO: File exists: /home/ubuntu/stanza_resources/en/lemma/genia.pt.
2021-03-21 13:57:24 INFO: File exists: /home/ubuntu/stanza_resources/en/depparse/genia.pt.
2021-03-21 13:57:25 INFO: File exists: /home/ubuntu/stanza_resources/en/ner/bionlp13cg.pt.
2021-03-21 13:57:25 INFO: File exists: /home/ubuntu/stanza_resources/en/pretrain/genia.pt.
2021-03-21 13:57:25 INFO: File exists

In [ ]:
nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})
doc = nlp("Spinal and bulbar muscular atrophy (SBMA) is an \
           inherited motor neuron disease caused by the expansion \
           of a polyglutamine tract within the androgen receptor (AR). \
           SBMA can be caused by this easily.")

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/ubuntu/.local/lib/python3.6/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/home/ubuntu/.local/lib/python3.6/site-packages/scispacy/c

In [ ]:
entity=doc.ents[1]
linker = nlp.get_pipe("scispacy_linker")
for umls_ent in entity._.kb_ents:
	print(linker.kb.cui_to_entity[umls_ent[0]])


CUI: C1839259, Name: Bulbo-Spinal Atrophy, X-Linked
Definition: An X-linked recessive form of spinal muscular atrophy. It is due to a mutation of the gene encoding the ANDROGEN RECEPTOR.
TUI(s): T047
Aliases (abbreviated, total: 39): 
	 Bulbospinal Muscular Atrophy, X linked, SMAX1, X Linked Spinal and Bulbar Muscular Atrophy, Bulbospinal muscular atrophy, kennedy's syndrome, Atrophy, Muscular, Spinobulbar, X Linked Bulbo Spinal Atrophy, X-Linked Spinal and Bulbar Muscular Atrophy, Bulbo Spinal Atrophy, X Linked, X-Linked Bulbo-Spinal Atrophy
CUI: C1947952, Name: anatomical bulb
Definition: A rounded dilation or expansion in a canal, vessel, or organ.
TUI(s): T017
Aliases: (total: 2): 
	 Bulbar, Bulb
CUI: C4083049, Name: Muscle (organ)
Definition: One of the contractile organs of the body.
TUI(s): T023
Aliases (abbreviated, total: 13): 
	 Muscles, NOS, Muscle organ, Myotendinous unit, Muscle, NOS, Muscle (organ), MUSCLE, Muscle, Muscles, Muscular, muscles organs


In [ ]:
entity._.kb_ents

[('C1839259', 0.7716761827468872),
 ('C1947952', 0.7680131196975708),
 ('C4083049', 0.7082285284996033)]

In [ ]:
from spacy import displacy
displacy.render(doc)
displacy.render(doc, style="ent")

[W109] Unable to save user hooks while serializing the doc. Re-add any required user hooks to the doc after processing.


In [ ]:
doc.to_json()

NameError: name 'doc' is not defined

In [ ]:
# JSL = {
#     "SENTENCE_RELATION": {description: "contradictions, entailments and premises & hypotheses"},
#     "CONTEXTUAL_WORD_MEANING": {description: "word meaning from context ([i beat him, the beat is good]-> false)"},
#     "SENTENCE_DETECTOR": {description: "split sentences in doc"},
#     "SENTENCE_SIMILARITY": ""
# }